# Outcome Prediction with RGCN Model

## 1. Load data from "processed_data" folder

In [1]:
import os
import time
import math
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
from torch.nn import Parameter

from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
from torch_geometric.utils import to_undirected

In [3]:
num_patients = 1000
inverse_triples=True
embed_dim = 32

entity = pd.read_csv(f'processed_data/sphn_entities_{num_patients}_noOutcome.tsv', sep='\t', index_col=0, header=None)
entity = entity.to_dict()[1]

indices = []
for i in range(num_patients):
    idx = f'<http://nvasc.org/synth_patient_{i}>'
    indices.append(entity[idx])
# events = pd.read_csv('../data/SPHN_events_noOutcome.tsv', sep='\t', header=None)
events = pd.read_csv(f'processed_data/sphn_events_{num_patients}_noOutcome.tsv', sep='\t', header=None)
y = joblib.load(f'../Data Generation/outcomes_{num_patients}_0.joblib')

non_valid_X, valid_X, non_valid_y, valid_y = train_test_split(indices, y, stratify=y, test_size=0.2)
train_X, testing_X, train_y, testing_y = train_test_split(non_valid_X, non_valid_y, stratify=non_valid_y, test_size=1./8)

# rus = RandomUnderSampler(random_state=0)
# under_idx, under_y = rus.fit_resample(np.asarray(train_X).reshape(-1, 1), np.asarray(train_y).reshape(-1,1))
# indices = np.asarray(indices)
# y = np.asarray(y)

edge_index = torch.vstack((torch.Tensor(events[0]).long(),torch.Tensor(events[2]).long()))
edge_type = torch.Tensor(events[1]).long()
train_idx = torch.Tensor(train_X).long().squeeze()
train_y = torch.Tensor(train_y).long()
val_idx = torch.Tensor(valid_X).long()
val_y = torch.Tensor(valid_y).long()
test_idx = torch.Tensor(testing_X).long()
test_y = torch.Tensor(testing_y).long()
num_nodes = len(entity)

if inverse_triples == True:
    edge_index = to_undirected(edge_index)
    edge_type = torch.cat((edge_type, edge_type))

data = Data(
    edge_index=edge_index,
    edge_type=edge_type,
    train_idx=train_idx,
    train_y=train_y,
    val_idx=val_idx,
    val_y=val_y,
    test_idx=test_idx,
    test_y=test_y,
    num_nodes=num_nodes,
)
embedding = Parameter(torch.empty(num_nodes, embed_dim))
torch.nn.init.xavier_uniform_(embedding, gain=math.sqrt(2.0))
data.x = embedding
data.num_relations = data.num_edge_types
data.num_classes = 3

In [4]:
print(data)

Data(edge_index=[2, 221870], edge_type=[221870], train_idx=[700], train_y=[700], val_idx=[200], val_y=[200], test_idx=[100], test_y=[100], num_nodes=33563, x=[33563, 32], num_relations=10, num_classes=3)


## 2. Training the model

In [9]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = RGCNConv(embed_dim, embed_dim, data.num_relations,
                          num_bases=4)
        self.conv2 = RGCNConv(embed_dim, data.num_classes, data.num_relations,
                          num_bases=4)

    def forward(self, edge_index, edge_type):
        x = F.relu(self.conv1(data.x, edge_index, edge_type))
        x = F.dropout(x, p=0.2)
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)


if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
# class_weight = torch.bincount(train_y).to(device)
# class_weight = class_weight / class_weight.max()

In [10]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y, 
                    #   weight=class_weight,
                      )
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    val_acc = float((pred[data.val_idx] == data.val_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, val_acc, test_acc

In [11]:
best_acc = 0.
times = []
for epoch in range(1, 1001):
    start = time.time()
    loss = train()
    if epoch % 50 == 0:
        train_acc, val_acc, test_acc = test()
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f} | ' 
            f'Train: {train_acc:.4f} '
            f'Valid: {val_acc:.4f} '
            f'Test: {test_acc:.4f}')    
        times.append(time.time() - start)
        # # Early stopping
        # if val_acc > best_acc:
        #     best_acc = val_acc
        #     patience = 10  # Reset patience counter
        # else:
        #     patience -= 1
        #     if patience == 0:
        #         print("Early stopping...")
        #         break
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 50, Loss: 0.9813 | Train: 0.5029 Valid: 0.4550 Test: 0.4300
Epoch: 100, Loss: 0.9423 | Train: 0.5500 Valid: 0.4600 Test: 0.4800
Epoch: 150, Loss: 0.9024 | Train: 0.6200 Valid: 0.4200 Test: 0.4600
Epoch: 200, Loss: 0.8458 | Train: 0.6371 Valid: 0.4450 Test: 0.4800
Epoch: 250, Loss: 0.7707 | Train: 0.6800 Valid: 0.4750 Test: 0.4500
Epoch: 300, Loss: 0.6911 | Train: 0.7314 Valid: 0.4850 Test: 0.4600
Epoch: 350, Loss: 0.5845 | Train: 0.7871 Valid: 0.4250 Test: 0.3900
Epoch: 400, Loss: 0.5061 | Train: 0.8243 Valid: 0.4500 Test: 0.4300
Epoch: 450, Loss: 0.4183 | Train: 0.8514 Valid: 0.4550 Test: 0.4200
Epoch: 500, Loss: 0.3616 | Train: 0.9071 Valid: 0.4800 Test: 0.3600
Epoch: 550, Loss: 0.2949 | Train: 0.9114 Valid: 0.4450 Test: 0.4000
Epoch: 600, Loss: 0.2348 | Train: 0.9443 Valid: 0.4700 Test: 0.3800
Epoch: 650, Loss: 0.2006 | Train: 0.9543 Valid: 0.4400 Test: 0.4000
Epoch: 700, Loss: 0.1875 | Train: 0.9686 Valid: 0.4250 Test: 0.4200
Epoch: 750, Loss: 0.1479 | Train: 0.9700 Valid: 0

In [8]:
from sklearn.metrics import roc_auc_score

model.eval()
with torch.no_grad():
    out = model(data.edge_index, data.edge_type).cpu()
    prob = F.softmax(out, dim=1)
auc = roc_auc_score(data.test_y.cpu(), prob[data.test_idx.cpu()], multi_class='ovr')
print(f'ROC AUC score: {auc:.4f}')
# print(prob.shape)

ROC AUC score: 0.4771
